# **The Storm Of the Century : Winds ERA5**
***

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
import metpy.calc as mpcalc
from scipy import ndimage  
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
import metpy
from metpy.plots import ctables

## **Date Input**

In [ ]:
startYear = 1993
startMonth = 3
startDay = 7
startHour = 0
startMinute = 0
startDateTime = dt(startYear,startMonth,startDay, startHour, startMinute)

endYear = 1993
endMonth = 3
endDay = 15
endHour = 0
endMinute = 0
endDateTime = dt(endYear,endMonth,endDay, endHour, endMinute)

In [ ]:
dateList = pd.date_range(startDateTime, endDateTime,freq="6H")
dateList

## **ERA5**

In [ ]:
reanalysis = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2', 
    consolidated=True
)

In [ ]:
reanalysis

## **Area and Domain**

In [ ]:
lonW = -99 + 360
lonE = -59 + 360
latS = 20
latN = 50
inc=.25
cLat, cLon = (latS + latN)/2, (lonW + lonE)/2
latRange = np.arange(latN,latS-inc,-inc) # expand the data range a bit beyond the plot range
lonRange = np.arange(lonW,lonE+inc,+inc) # Need to match longitude values to those of the coordinate variable
constrainLat, constrainLon = (0.6, 7)
proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree() # Our data is lat-lon; thus its native projection is Plate Carree.
res = '50m'

## **Variables**

In [ ]:
plevel=300
U = reanalysis['u_component_of_wind'].sel(time=dateList,level=plevel,latitude=latRange,longitude=lonRange)
V = reanalysis['v_component_of_wind'].sel(time=dateList,level=plevel,latitude=latRange,longitude=lonRange)
Z = reanalysis['geopotential'].sel(time=dateList,level=plevel,latitude=latRange,longitude=lonRange)

In [ ]:
lats = Z.latitude
lons = Z.longitude

## **MetPy Calculations**

In [ ]:
# Windspeed 
WSPD = mpcalc.wind_speed(U,V)
# Height
HGHT = mpcalc.geopotential_to_height(Z)
# U and V component
Ukts = U.metpy.convert_units('kts')
Vkts = V.metpy.convert_units('kts')

In [ ]:
WSPD = WSPD.metpy.convert_units('kts')
HGHT = HGHT.metpy.convert_units('dam')

In [ ]:
WSPD = WSPD.compute()
HGHT = HGHT.compute()

In [ ]:
U = U.compute()
V = V.compute()

## **Graphing**

In [ ]:
WSPD.min(),WSPD.max()

In [ ]:
HGHT.min(),HGHT.max()

In [ ]:
contour_int=10
minWSPD, maxWSPD = (100,210)
WSPDContours = np.arange(minWSPD, maxWSPD, contour_int)

In [ ]:
contour_int_HGHT=6
minHGHT,maxHGHT=(830,966)
HGHTContours=np.arange(minHGHT,maxHGHT,contour_int_HGHT)

In [ ]:
cmap='cool'
bounds = np.arange(50, 250, 25)
ticks = bounds

In [ ]:
levstr=f'{plevel}'

In [ ]:
for time in dateList:
    print("Processing", time)
    
    # Create the time strings, for the figure title as well as for the file name.
    timeStr = dt.strftime(time,format="%Y-%m-%d %H%M UTC")
    timeStrFile = dt.strftime(time,format="%Y%m%d%H")
    
    tl1 = str('ERA5, Valid at: '+ timeStr)
    tl2 = ("Winds (kts) and Geopotential Height (dam) at " + levstr + "mb")
    
    title_line = (tl1 + '\n' + tl2 + '\n')
    
    fig = plt.figure(figsize=(22,14)) # Increase size to adjust for the constrained lats/lons
    ax = plt.subplot(1,1,1,projection=proj_map)
    ax.set_extent ([lonW+constrainLon,lonE-constrainLon,latS+constrainLat,latN-constrainLat])
    ax.add_feature(cfeature.COASTLINE.with_scale(res))
    ax.add_feature(cfeature.STATES.with_scale(res),edgecolor='brown')
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature (cfeature.RIVERS.with_scale(res))
    ax.add_feature (cfeature.LAND.with_scale(res))
    ax.add_feature (cfeature.COASTLINE.with_scale(res))
    ax.add_feature (cfeature.LAKES.with_scale(res))
    ax.add_feature (cfeature.STATES.with_scale(res))
    ax.add_feature(cfeature.OCEAN)
    
 
    # Contour lines 
    cZ = ax.contour(lons, lats, HGHT.sel(time=time), levels=HGHTContours, colors='black', linewidths=1, transform=proj_data)
    ax.clabel(cZ, inline=1, fontsize=12, fmt='%.0f',colors='red',zorder=2)

    contourf=ax.contourf(lons,lats,WSPD.sel(time=time),levels=WSPDContours,cmap=cmap,transform=proj_data)
    cbar = plt.colorbar(contourf, ax=ax, ticks=ticks)
    cbar.set_label('Wind Speed kts',rotation=270, labelpad=15)
    
    # Wind barbs
    skip = 6
    ax.barbs(lons[::skip],lats[::skip],Ukts.sel(time=time)[::skip,::skip].values, Vkts.sel(time=time)[::skip,::skip].values, color='black',zorder=2,transform=proj_data,length=6)

    title = plt.title(title_line,fontsize=16)